    L6 提升

1、AdaBoost

提升二分类训练器

通过对原始数据加权，并不断优化权重，对弱分类器进行提升得到强分类器

强分类器是弱分类器的线性组合

In [80]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

导入数据 乳腺癌数据

In [81]:
cancer = load_breast_cancer()

In [84]:
print(cancer.data.shape)
print(cancer.feature_names)
print(set(cancer.target))

(569, 30)
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
{0, 1}


In [346]:
x_data = pd.DataFrame(cancer.data,columns = cancer.feature_names)[0:500]

In [348]:
y_data = cancer.target[0:500]

In [351]:
x_test = pd.DataFrame(cancer.data,columns = cancer.feature_names)[500:]

In [353]:
y_test = cancer.target[500:]

需把y_data中的0替换为-1以便后续计算需要

In [354]:
y_data[y_data==0] = -1.0
y_test[y_test==0] = -1.0

首先建立一个树桩

In [308]:
def split(data , feature , value , judge):
    
    #根据所给的特征和值对数据进行分类
    
    judgeArray = np.ones(data.shape[0])
    
    if judge == 'large':
        #如果比value大，则分为0，即没有癌症
        judgeArray[data[feature] > value] = -1.0
    
    else:
        #如果比value小，则分为0，即没有癌症
        judgeArray[data[feature] <= value] = -1.0
        
    
    return judgeArray

In [309]:
def Stump(x_data , y_data , D):
    
    #建立一个单层决策树，返回：分类器、误差率、分类结果
    
    #D是权重向量
    
    #初始化变量
    Stump = {}
    minErr = np.inf
    stepLen = 20
    bestClass = np.ones(len(y_data))
    
    #树桩
    for x_feature in x_data.columns:
        rangeMin = x_data[x_feature].min()
        rangeMax = x_data[x_feature].max()
        steps = ( rangeMax - rangeMin ) / stepLen #对于连续变量进行分段取值
        
        for step in range(-1,int(stepLen)+1):
            for judge in ['large', 'small']:
                value = rangeMin + float(step) * steps
                predictClass = split(x_data , x_feature , value , judge)
                errArray = np.ones(len(y_data))
                errArray[predictClass == y_data] = 0.0
                err = np.dot(D , errArray)
                
                if err < minErr:
                    minErr = err
                    bestClass = predictClass.copy()
                    
                    Stump['feature'] = x_feature
                    Stump['value'] = value
                    Stump['judge'] = judge
    
    return Stump , minErr , bestClass 

In [310]:
example1 , example2 , example3 = Stump(x_data , y_data , np.ones(len(y_data))/len(y_data))

In [311]:
example1

{'feature': 'worst perimeter', 'value': 110.64699999999999, 'judge': 'large'}

这样建立了一个单层决策树：当特征'worst perimeter'的值大于110.6469时预测为0，即没有癌症

In [342]:
def AdaBoostDt(x_data , y_data , iterNum):
    
    #用adaboost提升单层决策树
    
    ClassTree = []
    D = np.ones(len(y_data))/len(y_data)
    weighBestClass = np.zeros(len(y_data))
    i = 1
    
    while 1:
        bestStump , error , bestClass = Stump(x_data , y_data , D)
        α = 1/2 * np.log((1 - error) / (error + 1e-10))
        bestStump['α'] = α
        ClassTree.append(bestStump)
        D = D * np.exp(-α * y_data * bestClass)\
            / np.sum(D * np.exp(-α * y_data * bestClass))
        
        #计算分类器的总误差
        
        weighBestClass += α * bestClass
        grossErr = np.dot(np.sign(weighBestClass) != y_data ,
                          np.ones(len(y_data))/len(y_data))
        i += 1
        if grossErr == 0.0:
            break
        
        if i >= iterNum:
            break
    
    return AdaClassTree

In [355]:
AdaTree = AdaBoostDt(x_data , y_data , 2000)

In [357]:
len(AdaTree)

45

然后用训练好的强分类器进行预测

In [365]:
def AdaForecast(x_test , AdaClassTree):
    
    #用分类器进行分类
    
    f = np.zeros(x_test.shape[0])
    
    for i in range(len(AdaClassTree)):
        ClassOutPut = split(x_test , AdaClassTree[i]['feature'],
                  AdaClassTree[i]['value'] , AdaClassTree[i]['judge'])
        f += AdaClassTree[i]['α'] * ClassOutPut
        G = np.sign(f)
    
    return G

输出预测结果：

In [366]:
result1 = AdaForecast(x_test , AdaTree)

预测误差率：

In [368]:
np.dot(np.sign(result1) != y_test ,np.ones(len(y_test))/len(y_test))

0.028985507246376812